In [2]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence

from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [3]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [4]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])


In [5]:
def gaussian_reader(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df

In [6]:
def chi2bool(df):
#returns a tuple(chi, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    empty_5 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
    for x,y,i in zip(empty_1, empty_2, empty):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=True) #returns chi, p_value, dof
        empty_3.append([i , chisquare])
    for value in range(len(empty_3)):
        empty_4.append(value)
    for x,y in zip(empty_4, empty_3):
        empty_5.append([x,y])
    return empty_5


In [7]:
def chi2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    empty_5 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
    for x,y,i in zip(empty_1, empty_2, empty):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_3.append([i , chisquare])
    for value in range(len(empty_3)):
        empty_4.append(value)
    for x,y in zip(empty_4, empty_3):
        empty_5.append([x,y])
    return empty_5


In [ ]:
def chi2return(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    empty_5 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
    for x,y,i in zip(empty_1, empty_2, empty):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_3.append([i , chisquare])
    for value in range(len(empty_3)):
        empty_4.append(value)
    for x,y in zip(empty_4, empty_3):
        empty_5.append([x,y])
    return empty_5


In [9]:
df = csv_format_discrete(r'C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat')

In [15]:
v = list(df)
empty = []
empty_1 = []
empty_2 = []
empty_3 = []
empty_4 = []
empty_5 = []
itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
for i in itertools_combinations:
    empty_1.append(i[0])
    empty_2.append(i[1])
for t in range(len(itertools_combinations)):
    empty.append(itertools_combinations[t])
for x,y,i in zip(empty_1, empty_2, empty):
    chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=True) #returns chi, p_value, dof
    empty_3.append([i , chisquare])
for value in range(len(empty_3)):
    empty_4.append(value)
for x,y in zip(empty_4, empty_3):
    empty_5.append([x,y])

In [36]:
empty_3[0]

[('PULMEMBOLUS', 'PAP'), False]

In [50]:
k = np.array(empty_3)
np.where('True')

C:\Users\User\AppData\Local\Temp\ipykernel_2240\1681355355.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  k = np.array(empty_3)


(array([0], dtype=int64),)

In [68]:
A

[[('PULMEMBOLUS', 'PAP'), False], [('PULMEMBOLUS', 'INTUBATION'), True]]

In [102]:
test_list = []
A = [empty_3[0], empty_3[2], empty_3[3]]
for i in A:
    for k in i:
        test_list.append(k)

In [140]:
A

[[('PULMEMBOLUS', 'PAP'), False],
 [('PULMEMBOLUS', 'INTUBATION'), True],
 [('PULMEMBOLUS', 'MINVOLSET'), True]]

In [142]:
word = []
for element in A:
    for j in element:
        for word in j.split():
            print(word)

AttributeError: 'tuple' object has no attribute 'split'

In [129]:
if [] in A:
    print('athis')
else:
    print('that')

that


In [69]:
empty_3

[[('PULMEMBOLUS', 'PAP'), False],
 [('PULMEMBOLUS', 'KINKEDTUBE'), True],
 [('PULMEMBOLUS', 'INTUBATION'), True],
 [('PULMEMBOLUS', 'MINVOLSET'), True],
 [('PULMEMBOLUS', 'VENTMACH'), True],
 [('PULMEMBOLUS', 'DISCONNECT'), True],
 [('PULMEMBOLUS', 'VENTTUBE'), True],
 [('PULMEMBOLUS', 'VENTLUNG'), True],
 [('PULMEMBOLUS', 'SHUNT'), False],
 [('PULMEMBOLUS', 'VENTALV'), True],
 [('PULMEMBOLUS', 'FIO2'), True],
 [('PULMEMBOLUS', 'PVSAT'), True],
 [('PULMEMBOLUS', 'SAO2'), False],
 [('PULMEMBOLUS', 'INSUFFANESTH'), True],
 [('PULMEMBOLUS', 'ARTCO2'), True],
 [('PULMEMBOLUS', 'ANAPHYLAXIS'), True],
 [('PULMEMBOLUS', 'TPR'), True],
 [('PULMEMBOLUS', 'CATECHOL'), True],
 [('PULMEMBOLUS', 'HR'), True],
 [('PULMEMBOLUS', 'ERRCAUTER'), True],
 [('PULMEMBOLUS', 'HREKG'), True],
 [('PULMEMBOLUS', 'HYPOVOLEMIA'), True],
 [('PULMEMBOLUS', 'LVFAILURE'), True],
 [('PULMEMBOLUS', 'LVEDVOLUME'), True],
 [('PULMEMBOLUS', 'CVP'), True],
 [('PULMEMBOLUS', 'STROKEVOLUME'), True],
 [('PULMEMBOLUS', 'CO'), 

In [53]:
A = np.array([True, False, True, False, True])
# true_list = A[A == True].index.tolist()
np.where(A)


(array([0, 2, 4], dtype=int64),)

In [18]:
itertools_combinations

[('PULMEMBOLUS', 'PAP'),
 ('PULMEMBOLUS', 'KINKEDTUBE'),
 ('PULMEMBOLUS', 'INTUBATION'),
 ('PULMEMBOLUS', 'MINVOLSET'),
 ('PULMEMBOLUS', 'VENTMACH'),
 ('PULMEMBOLUS', 'DISCONNECT'),
 ('PULMEMBOLUS', 'VENTTUBE'),
 ('PULMEMBOLUS', 'VENTLUNG'),
 ('PULMEMBOLUS', 'SHUNT'),
 ('PULMEMBOLUS', 'VENTALV'),
 ('PULMEMBOLUS', 'FIO2'),
 ('PULMEMBOLUS', 'PVSAT'),
 ('PULMEMBOLUS', 'SAO2'),
 ('PULMEMBOLUS', 'INSUFFANESTH'),
 ('PULMEMBOLUS', 'ARTCO2'),
 ('PULMEMBOLUS', 'ANAPHYLAXIS'),
 ('PULMEMBOLUS', 'TPR'),
 ('PULMEMBOLUS', 'CATECHOL'),
 ('PULMEMBOLUS', 'HR'),
 ('PULMEMBOLUS', 'ERRCAUTER'),
 ('PULMEMBOLUS', 'HREKG'),
 ('PULMEMBOLUS', 'HYPOVOLEMIA'),
 ('PULMEMBOLUS', 'LVFAILURE'),
 ('PULMEMBOLUS', 'LVEDVOLUME'),
 ('PULMEMBOLUS', 'CVP'),
 ('PULMEMBOLUS', 'STROKEVOLUME'),
 ('PULMEMBOLUS', 'CO'),
 ('PULMEMBOLUS', 'BP'),
 ('PULMEMBOLUS', 'EXPCO2'),
 ('PULMEMBOLUS', 'ERRLOWOUTPUT'),
 ('PULMEMBOLUS', 'PRESS'),
 ('PULMEMBOLUS', 'HRBP'),
 ('PULMEMBOLUS', 'MINVOL'),
 ('PULMEMBOLUS', 'HISTORY'),
 ('PULMEMBOLUS',